In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import rasterio
import geojson
import firedrake
import icepack, icepack.plot, icepack.models

# Helheim Glacier

In this demo we'll look at Helheim Glacier in southeast Greenland.
Helheim is one of the bigger glaciers draining the Greenland Ice Sheet by mass flux along with Jakobshavn, Kangerdlugssuaq, Petermann, and the Northeast Greenland Ice Stream.
Much of the procedure for working with real data should be familiar from the demo on [inverse problems](https://icepack.github.io/icepack.demo.04-ice-shelf-inverse.html), where we inferred the fluidity of the Larsen C Ice Shelf in the Antarctic Peninsula.
Here we'll use many of the same techniques, only instead of inferring the fluidity, we'll look at the friction coefficient for glacier sliding over the underlying bedrock.
In the following, we'll assume that ice flow is by Weertman sliding:

$$\tau_b = -C|u|^{\frac{1}{m} - 1}u,$$

where $\tau_b$ is the basal shear stress, $u$ is the sliding velocity, $m \approx 3$ is the friction exponent, and $C$ is the friction coefficient.
First, we'll estimate the friction coefficient using a 2D, depth-averaged model.
Then we'll use this solution as an initial guess using the 3D hybrid model that we showed in [this demo](https://icepack.github.io/icepack.demo.05-hybrid-ice-stream.html) for a synthetic ice stream.

### Input data

Loading in the mesh and input data should be mostly familiar from the previous demos on Larsen C.
We'll be using different data sets since we're looking at Greenland rather than Antarctica this time around.

In [ ]:
outline_filename = icepack.datasets.fetch_helheim_outline()
with open(outline_filename, 'r') as outline_file:
    outline = geojson.load(outline_file)

geometry = icepack.meshing.collection_to_geo(outline, lcar=5e3)
with open('helheim.geo', 'w') as geo_file:
    geo_file.write(geometry.get_code())

In [ ]:
!gmsh -2 -format msh2 -v 2 -o helheim.msh helheim.geo

The initial mesh that we've generated is pretty coarse, so to get finer resolution we'll create a hierarchy of refined meshes.
In our case the mesh hierarchy will only go one level deep, so each triangle has been cut into four similar triangles.
Where possible, it's best to start with the coarsest mesh and refine up to the level you need; you can always make a small problem bigger, but it's hard to make a big problem smaller.

In [ ]:
coarse_mesh = firedrake.Mesh('helheim.msh')
mesh_hierarchy = firedrake.MeshHierarchy(coarse_mesh, 1)
mesh = mesh_hierarchy[-1]

In [ ]:
fig, axes = icepack.plot.subplots()
icepack.plot.triplot(mesh, axes=axes)
axes.legend()

In [ ]:
thickness_filename = icepack.datasets.fetch_bedmachine_greenland()
thickness = rasterio.open('netcdf:' + thickness_filename + ':thickness', 'r')
surface = rasterio.open('netcdf:' + thickness_filename + ':surface', 'r')

Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)
h0 = icepack.interpolate(thickness, Q)
s0 = icepack.interpolate(surface, Q)

from firedrake import inner, grad, dx
def smooth(q0, α):
    q = q0.copy(deepcopy=True)
    J = 0.5 * ((q - q0)**2 + α**2 * inner(grad(q), grad(q))) * dx
    F = firedrake.derivative(J, q)
    firedrake.solve(F == 0, q)
    return q

α = firedrake.Constant(2e3)
h = smooth(h0, α)
s = smooth(s0, α)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(h, 40, axes=axes)
fig.colorbar(contours)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(s, 40, axes=axes)
fig.colorbar(contours)

In [ ]:
velocity_filenames = icepack.datasets.fetch_measures_greenland()
velocity_dict = {
    key: [f for f in velocity_filenames if key in f][0]
    for key in ['vx', 'vy', 'ex', 'ey']
}
vx = rasterio.open(velocity_dict['vx'], 'r')
vy = rasterio.open(velocity_dict['vy'], 'r')
ex = rasterio.open(velocity_dict['ex'], 'r')
ey = rasterio.open(velocity_dict['ey'], 'r')

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, family='CG', degree=2)
u_obs = icepack.interpolate((vx, vy), V)
σx = icepack.interpolate(ex, Q)
σy = icepack.interpolate(ey, Q)

In [ ]:
fig, axes = icepack.plot.subplots()
σ = firedrake.interpolate(firedrake.sqrt(σx**2 + σy**2), Q)
levels = np.linspace(0, 50, 26)
contours = icepack.plot.tricontourf(σ, levels=levels, extend='max', axes=axes)
fig.colorbar(contours)

Now that we've loaded in the ice thickness and surface elevation, we'll calculate the gravitational driving stress

$$\tau_d = -\rho_Igh\nabla s.$$

The gravitational driving stress is in balance with internal viscous stresses and with basal stress.
By knowing roughly what the magnitudes of the driving stress and the velocity are, we can get a better idea of what a reasonable starting value of the friction coefficient should be.

In [ ]:
from firedrake import grad
from icepack.constants import ice_density as ρ_I, gravity as g
τ = firedrake.project(-ρ_I * g * h * grad(s), V)

In [ ]:
import numpy as np
fig, axes = icepack.plot.subplots()
levels = np.linspace(0, .5, 51)
contours = icepack.plot.tricontourf(τ, levels=levels, extend='max', axes=axes)
fig.colorbar(contours, label='megapascals')

To initialize the inverse problem, we'll assume that the basal shear stress takes up half of the driving stress.
This is a totally ad hoc assumption and it's not obvious a priori that this is going to give us reasonable values of the ice velocity.
Nonetheless, it happens to work well enough as a starting point.
We'll also need to re-parameterize the basal shear stress in terms of some auxiliary variable $\theta$ in order to guarantee that the friction coefficient is strictly positive.
In the following, we'll use

$$\tau_b = -\frac{\tau_0}{u_0^{1/m}}e^{-\phi}|u|^{\frac{1}{m} - 1}u$$

where $\tau_0$ and $u_0$ are the average magnitudes of the driving stress and speed respectively.

In [ ]:
from firedrake import exp, ln, sqrt, assemble
from icepack.constants import weertman_sliding_law as m
import icepack.models.friction

u = u_obs.copy(deepcopy=True)
speed = sqrt(inner(u, u))
stress = sqrt(inner(τ, τ))

area = assemble(firedrake.Constant(1) * dx(mesh))
speed_avg = assemble(speed * dx) / area
stress_avg = assemble(stress * dx) / area

print('Average speed:  {} meters / year'.format(speed_avg))
print('Average stress: {} kilopascals'.format(1e3 * stress_avg))

fraction = 0.5
C = fraction * stress / speed**(1/m)
q = firedrake.interpolate(-ln(speed_avg**(1/m) * C / stress_avg), Q)

def bed_friction(u, q):
    C = stress_avg / speed_avg**(1/m) * exp(-q)
    return icepack.models.friction.bed_friction(u, C)

For now, we'll assume that the ice is at a constant -13C.
We'll revisit this assumption using a heat flow model later.

In [ ]:
T = firedrake.Constant(260.)
A = icepack.rate_factor(T)

In [ ]:
ice_stream = icepack.models.IceStream(friction=bed_friction)
opts = {'dirichlet_ids': [1, 2, 4, 5, 6], 'tol': 1e-6}
u = ice_stream.diagnostic_solve(u0=u_obs, h=h, s=s, A=A, q=q, **opts)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(u, axes=axes)
fig.colorbar(contours, label='meters / year')

### Inferring the friction

Next we'll apply a similar procedure as the first demo on inverse problems to back out the friction coefficient.
The first ingredient is to define the objective and regularization functionals.

In [ ]:
def objective(u):
    δu = u - u_obs
    return 0.5 * ((δu[0] / σx)**2 + (δu[1] / σy)**2) * dx

Φ = firedrake.Constant(3.)
L = firedrake.Constant(19.5e3)
def regularization(q):
    return 0.5 * (L / Φ)**2 * inner(grad(q), grad(q)) * dx

In [ ]:
import icepack.inverse

problem = icepack.inverse.InverseProblem(
    model=ice_stream,
    method=icepack.models.IceStream.diagnostic_solve,
    objective=objective,
    regularization=regularization,
    state_name='u',
    state=u,
    parameter_name='q',
    parameter=q,
    model_args={'h': h, 's': s, 'u0': u, 'A': A, 'tol': 1e-6},
    dirichlet_ids=opts['dirichlet_ids']
)

In [ ]:
def print_error_and_regularization(solver):
    E = firedrake.assemble(solver.objective)
    R = firedrake.assemble(solver.regularization)
    print('{:g}, {:g}'.format(E/area, R/area))

In [ ]:
solver = icepack.inverse.GaussNewtonSolver(problem, print_error_and_regularization)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(solver.search_direction, axes=axes)
fig.colorbar(contours)

In [ ]:
iterations = solver.solve(
    rtol=5e-3,
    atol=0.0,
    max_iterations=30
)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(solver.parameter, 40, axes=axes)
fig.colorbar(contours)

In [ ]:
q = solver.parameter
u = solver.state

C = stress_avg / speed_avg**(1/m) * exp(-q)
U = sqrt(inner(u, u))
τ_b = firedrake.interpolate(C * U**(1/m), Q)

In [ ]:
fig, axes = icepack.plot.subplots()
levels = np.linspace(0, 0.5, 51)
contours = icepack.plot.tricontourf(τ_b, levels=levels, extend='max', axes=axes)
fig.colorbar(contours, label='megapascals')

### With the hybrid model

The model that we've used to infer the basal shear stress assumes that almost none of the ice flow is by vertical shear, but that might not be a good assumption in slower-flowing parts of the domain.
To get a better estimate, we can try again using the hybrid model.
We can cut down on the computational cost of using the more expensive hybrid model by initializing it with the estimates from the 2D model.

We showed how to use the hybrid model directly in the previous demo, but we didn't show how to take 2D model results and lift them into 3D.
Icepack provides a convenience function `lift3d` to do just that.

In [ ]:
help(icepack.lift3d)

First, we'll extrude the mesh and create a function space to represent scalar fields that are constant throughout the whole column.
Here we'll use the `R` space in the vertical, which represents spatially constant functions (see also [this page](https://firedrakeproject.org/r-space.html#using-r-space-with-extruded-meshes) from the Firedrake documentation).

In [ ]:
mesh3d = firedrake.ExtrudedMesh(mesh, 1)
Qc = firedrake.FunctionSpace(
    mesh3d, family='CG', degree=2, vfamily='R', vdegree=0
)
q = icepack.lift3d(solver.parameter, Qc)
h = icepack.lift3d(h, Qc)
s = icepack.lift3d(s, Qc)
σx = icepack.lift3d(σx, Qc)
σy = icepack.lift3d(σy, Qc)

Next, we need to lift the 2D depth-averaged velocity into 3D.
This takes place in two stages, the first of which is to extrude the velocity to be constant in the vertical direction.
The initial function space `V0` will use degree-0 elements in the vertical.

In [ ]:
V0 = firedrake.VectorFunctionSpace(
    mesh3d, dim=2, family='CG', degree=2, vfamily='R', vdegree=0
)
u0 = icepack.lift3d(solver.state, V0)
u_obs = icepack.lift3d(u_obs, V0)

Next, we'll add vertical variation; the function space `V` has degree 2 in the vertical.

In [ ]:
V = firedrake.VectorFunctionSpace(
    mesh3d, dim=2, family='CG', degree=2, vfamily='GL', vdegree=2
)

x, y, ζ = firedrake.SpatialCoordinate(mesh3d)
u_initial = firedrake.interpolate((1 - (1 - ζ)**2) * u0, V)

Finally, we'll create a model object.

In [ ]:
def bed_friction(u, q):
    C = stress_avg / speed_avg**(1/m) * exp(-q)
    return icepack.models.hybrid.bed_friction(u, C)

model = icepack.models.HybridModel(friction=bed_friction)

In [ ]:
u = model.diagnostic_solve(u0=u_initial, h=h, s=s, A=A, q=q, **opts)

In [ ]:
fig, axes = icepack.plot.subplots()
contours = icepack.plot.tricontourf(u, axes=axes)
fig.colorbar(contours)

**Well bummer**.
The following code is too slow to be useful.
Time to profile.

In [ ]:
from firedrake import ds_b, ds_t
def objective(u):
    δu = u - u_obs
    return 0.5 * ((δu[0] / σx)**2 + (δu[1] / σy)**2) * ds_t

Φ = firedrake.Constant(3.)
L = firedrake.Constant(19.5e3)
def regularization(q):
    return 0.5 * (L / Φ)**2 * inner(grad(q), grad(q)) * ds_b

problem = icepack.inverse.InverseProblem(
    model=model,
    method=type(model).diagnostic_solve,
    objective=objective,
    regularization=regularization,
    state_name='u',
    state=u,
    parameter_name='q',
    parameter=q,
    model_args={'h': h, 's': s, 'u0': u, 'A': A, 'tol': 1e-6},
    dirichlet_ids=opts['dirichlet_ids']
)

In [ ]:
solver = icepack.inverse.GaussNewtonSolver(problem, print_error_and_regularization)

In [ ]:
iterations = solver.solve(rtol=5e-3, atol=0., max_iterations=30)